<a href="https://colab.research.google.com/github/Mohammadhsiavash/DeepL-Training/blob/main/AI%20Agents%20%2B%20Automation/07_Browser_Agent_with_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create an AI agent that:
1. Reads web pages using a browser or web scraper
2. Stores key information in memory (e.g., vector database or list)
3. Answers questions based on what it has read
4. Remembers past content and avoids re-reading the same

Agent Patern: ReAct + Memory + Web Access
Tools used:

WebReaderTool : reads/scrapes webpages

MemoryTool : stores and recalls relevant passages

QA Agent : uses context to answer questions


In [1]:
!pip install requests beautifulsoup4 chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 76.6 MB/s eta 0:0

# Create Web Reader Tool

In [2]:
import requests
from bs4 import BeautifulSoup

def read_webpage(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  return soup.get_text()


# Set Up a Vector Memory with ChromaDB

In [3]:
import chromadb
from sentence_transformers import SentenceTransformer

chroma_client = chromadb.Client()
collection = chroma_client.create_collection("browser_memory")
embedder = SentenceTransformer('all-MiniLM-L6-v2')


def memorize(text, source):
  chunks = [text[i:i+500] for i in range(0, len(text), 500)]
  embeddings = embedder.encode(chunks)
  for i, chunk in enumerate(chunks):
    collection.add(
    documents=[chunk],
    embeddings=[embeddings[i]],
    ids=[f"{source}-{i}"]
    )

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Ask Questions Based on Memory

In [4]:
def answer_question(query):
  query_emb = embedder.encode(query)
  results = collection.query(
    query_embeddings=[query_emb],
    n_results=3
  )
  context = "\n\n".join(results['documents'][0])
  return context # This is your context to send to an LLM

# Connect to an LLM (OpenAI or Local)

In [5]:
from transformers import pipeline
qa_llm = pipeline("text-generation", model="gpt2")

def ask_agent(question):
  context = answer_question(question)
  prompt = f"""You are a research assistant.
  Context:
  {context}
  Question: {question}
  Answer:"""
  return qa_llm(prompt, max_new_tokens=100)[0]['generated_text']

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


# Run the Agent End-to-End

In [6]:
url = "https://en.wikipedia.org/wiki/Artificial_intelligence"
text = read_webpage(url)
memorize(text, "AI_Wikipedia")
response = ask_agent("What is the history of artificial intelligence?")
print(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You are a research assistant.
  Context:
  Please set a user-agent and respect our robot policy https://w.wiki/4wJS. See also T400119.

  Question: What is the history of artificial intelligence?
  Answer:  This is a collection of topics that I have written about in the past.  I am not commenting on specific topics. This is not a complete history of AI. I have researched AI for many years and I have created many interesting things. I do not intend to use AI to solve problems for anyone or anyone's benefit. I am not going to try to cover every problem I know or think I know. I will just try to explore everything. This is a very interesting topic.
 
